In [1]:
import pandas as pd
import random
random.seed(101)

In [2]:
df = pd.read_csv("https://open.canada.ca/data/dataset/0797e893-751e-4695-8229-a5066e4fe43c/resource/19383ca2-b01a-487d-88f7-e1ffbc7d39c2/download/ati.csv")
dispos = df['disposition']

In [3]:
#let's try to fix the data by using a classifier
print("There are "+str(len(dispos.unique()))+" unique dispositions")
dispos.sample(5)

There are 609 unique dispositions


25849                                  All disclosed
9264                               Disclosed in part
373                                Disclosed in Part
5707          No Record exist / Aucun enregistrement
14870    Disclosed in part / Communication partielle
Name: disposition, dtype: object

In [4]:
#we can do some cleaning
def cleaner(text):
    text = str(text)
    text = text.lower()
    text = text.split("/")[0]
    text = text.split("\n")[0]
    text = text.split("\\")[0]
    text = text.split("\t")[0]
    text = text.strip()
    #text = translate_to_en(text)
    #text = translator.translate(text)
    return str(text)

def translate_to_en(blob): #finds the blob and translates it to english if it isn't already. 
    #Google translate will not let us do many of these in a row.
    if type(blob) != 'textblob.blob.TextBlob':
            blob = TextBlob(blob)
    if (blob.detect_language() != 'en'):
        return blob.translate(to='en')
    else:
        return blob


In [5]:
dispos = pd.DataFrame(dispos, columns=['disposition'])
dispos['disposition_clean'] = dispos['disposition'].apply(lambda row: cleaner(row))
#unique_df['disposition_clean'] = unique_df['disposition_clean'].apply(lambda row: translate_to_en(row)) # can't do this, google translate is rate limiting.
unique = dispos.disposition_clean.unique()
print("There are "+str(len(unique))+" unique responses in the disposition column, after cleaning")

There are 181 unique responses in the disposition column, after cleaning


In [6]:
with open('sample_dispo.txt', 'w') as f: #save the dispositions in a file
    for item in unique:
        f.write("%s\n" % item)

In [7]:
#manually labelled the dispositions and read back in
manual = pd.read_csv("dispo_manual.csv")

In [8]:
manual.dropna(inplace=True)

In [9]:
df['clean_dispo'] = df['disposition'].apply(lambda row: cleaner(row))

In [10]:
sm = df[['year','month','summary_en', 'disposition','pages','owner_org','clean_dispo']]

In [11]:
sm.head()

,year,month,summary_en,disposition,pages,owner_org,clean_dispo
0,2019,7,A copy of the following briefing notes: 248350...,Disclosed in part/\r\nCommunication partielle,21.0,aafc-aac,disclosed in part
1,2019,7,A copy of the following briefing notes: 248665...,Disclosed in part/\r\nCommunication partielle,29.0,aafc-aac,disclosed in part
2,2019,7,A copy of the following briefing notes: 248710...,Disclosed in part/\r\nCommunication partielle,15.0,aafc-aac,disclosed in part
3,2019,8,A copy of the following briefing notes: 247950...,Disclosed in part/\r\nCommunication partielle,168.0,aafc-aac,disclosed in part
4,2019,8,A copy of the following briefing notes: 247938...,Disclosed in part/\r\nCommunication partielle,306.0,aafc-aac,disclosed in part


In [12]:
merge = pd.merge(sm, manual, left_on = 'clean_dispo',right_on = 'disposition',)

In [13]:
merge.code.unique()

array(['disclosed in part', 'all disclosed', 'no records exist',
       'all excluded', 'all exempted', 'neither confirmed nor denied',
       'request abandoned', 'error', 'request transferred'], dtype=object)

In [14]:
merge.shape

(29663, 9)

In [15]:
merge.to_csv("../data/clean.csv", index = False)

In [ ]:
#Questions
#Can we predict the # of pages?
#can we predict the result of the request(disposition)?
